In [ ]:
import tensorflow as tf
import numpy as np, h5py
#import scipy.io as sio
import sys
import random
#import kNN_cosine
import re
from numpy import *  
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam
from keras.models import load_model,Model

import matplotlib.pyplot as plt
from sklearn import manifold
from sklearn.cluster import KMeans
from numpy import *
import operator

In [ ]:
# 1. 加载数据
# x_train_feature = np.load('./datas/train/x_train.npy',allow_pickle=True)
x_train = np.load('./datas/train/x_train.npy').astype('float32')/255.
y_train = np.load('./datas/train/y_train.npy')

x_test = np.load('./datas/test/x_test.npy').astype('float32')/255.
y_test = np.load('./datas/test/y_test.npy')

y_tra = np_utils.to_categorical(y_train,num_classes=3)
y_tes = np_utils.to_categorical(y_test,num_classes=4)

In [ ]:
# 定义顺序模型
model = Sequential()

# 第一个卷积层
# input_shape 输入平面
# filters 卷积核/滤波器个数
# kernel_size 卷积窗口大小
# strides 步长
# padding padding方式 same/valid
# activation 激活函数
model.add(Convolution2D(
    input_shape = (64,64,1),
    filters = 32,
    kernel_size = 5,
    strides = 1,
    padding = 'same',
    activation = 'relu',
))
# 第一个池化层
model.add(MaxPooling2D(
    pool_size = 2,
    strides = 2,
    padding = 'same',
))
# 第二个卷积层
model.add(Convolution2D(64,5,strides=1,padding='same',activation='relu'))
# 第二个池化层
model.add(MaxPooling2D(2,2,'same'))
# 把第二个池化层的输出扁平化为1维
model.add(Flatten())
# 第一个全连接层
model.add(Dense(4096,activation='relu'))
model.add(Dense(2048,activation='relu'))
model.add(Dense(2048,activation='relu'))



# Dropout
model.add(Dropout(0.25))
# 第二个全连接层
model.add(Dense(3,activation='softmax'))

# 定义优化器
adam = Adam(lr=1e-4)

# 定义优化器,loss function,训练过程中计算准确率
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

# 训练模型
model.fit(x_train,y_tra,batch_size=64,epochs=25)

# 评估模型
loss,accuracy = model.evaluate(x_train,y_tra)


#保存模型
model.save('cnn_extractor.h5')
print(model.summary())
print('train loss',loss)
print('train accuracy',accuracy)

In [ ]:
#x_train = np.load('./data/train/x_train.npy')
# x_train = x_train.reshape(-1,64,64,1)/255.0

#载入模型
model = load_model('cnn_extractor.h5')
resnet_model = Model(inputs=model.input, outputs=model.get_layer('dense_3').output)
intermediate_output1 = resnet_model.predict(x_train)
print(intermediate_output1.shape)
#print(intermediate_output1)
np.save('./datas/features/feature_train', intermediate_output1 )


# x_test = np.load('./data/test/x_test.npy')
# x_test = x_test.reshape(-1,64,64,1)/255.0
#载入模型
model = load_model('cnn_extractor.h5')
resnet_model = Model(inputs=model.input, outputs=model.get_layer('dense_3').output)
intermediate_output2 = resnet_model.predict(x_test)
print(intermediate_output2.shape)
np.save('./datas/features/feature_test', intermediate_output2 )